View PDF

The following code allows the reading of a PDF file and makes it easier to obtain information from it, by asking questions through a chat (Chatgpt)

1. Configuration

In [1]:
%pip install openai
%pip install langchain
%pip install faiss-cpu
%pip install streamlit
%pip install PyPDF2
%pip install sentence-transformers


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the 

In [33]:
import streamlit as st
import requests
import logging

import os
from PyPDF2 import PdfReader
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from sentence_transformers import SentenceTransformer
from langchain.callbacks import get_openai_callback
from langchain.prompts.chat import ChatPromptTemplate
#from langchain.document_loaders import PyPDFLoader
#from langchain.document_loaders import Docx2txtLoader

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)

2. Read PDF

The loading and reading of the file is defined 

In [4]:
"""def load_file(file):
    name, ext = os.path.splitext(file)
    if ext == ".pdf":
        loader = PyPDFLoader(file)
    elif ext == ".docx":
        print(f'Loader {file}...')
        loader = Docx2txtLoader(file)
    else:
        print('The type file is not support')
        return None
    data = loader.load()
    return data"""
#file pdf
URL = 'https://revistas.pedagogica.edu.co/index.php/PYS/article/download/6006/4980/15252'
doc_to_download = requests.get(URL)

#save file
pdf_file = open('cultura_colombiana.pdf', 'wb')
pdf_file.write(doc_to_download.content)
logging.info('loaded file')

INFO:root:loaded file


In [5]:
pdf_file_obj = open(file='cultura_colombiana.pdf', mode='rb')
pdf_doc = PdfReader(pdf_file_obj)
logging.info(pdf_doc)

INFO:root:<PyPDF2._reader.PdfReader object at 0x1315a8110>


We obtain the number of pages or resulting documents

In [6]:

text=""
for page in pdf_doc.pages:
    text += page.extract_text()
    logging.info(text)


INFO:root:Pedagogía y Saberes Nº 15  2000Silvio Aristizábal Giraldo
Profesor Universidad Pedagógica NacionalLA DIVERSIDAD ÉTNICA Y CULTURAL DE COLOMBIA:
UN DESAFÍO PARA LA EDUCACIÓN
1. DIVERSIDAD ÉTNICA
Y CULTURAL
La mayoría de los colombianos
cuando escucha hablar de la di-versidad étnica y cultural de lanación piensa que esta expre-sión se refiere sólo a la existen-cia de indígenas y afrocolom-bianos. Pocos entienden que laexpresión alude en algún senti-do a las diferencias entre un cos-teño, un valluno y un llanero.Muchos consideran que entrelos indígenas no hay diferenciasy para otros,  los raizales de SanAndrés y Providencia no se dis-tinguen de los chocoanos. Sinembargo, la realidad es bien di-ferente. Veamos un poco cómoes la situación:
1.1. Pueblos IndígenasExisten en el país más de 80
Pueblos Indígenas que hablan66 lenguas. Cada uno de estospueblos tiene su cosmovisión,sus códigos diferentes de signi-ficación de la realidad, sus valo-res, creencias, costumbres y ex-presiones a

In [7]:
text[500:1000]

' y un llanero.Muchos consideran que entrelos indígenas no hay diferenciasy para otros,  los raizales de SanAndrés y Providencia no se dis-tinguen de los chocoanos. Sinembargo, la realidad es bien di-ferente. Veamos un poco cómoes la situación:\n1.1. Pueblos IndígenasExisten en el país más de 80\nPueblos Indígenas que hablan66 lenguas. Cada uno de estospueblos tiene su cosmovisión,sus códigos diferentes de signi-ficación de la realidad, sus valo-res, creencias, costumbres y ex-presiones artísticas '

3. Create Chunks

divide the texts into different Chunks

We try to divide at paragraph level, then at sentence level and finally at word level.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)
logging.info(text_splitter)

INFO:root:<langchain.text_splitter.RecursiveCharacterTextSplitter object at 0x10b19b790>


In [9]:
chunk = text_splitter.split_text(text=text)
logging.info(chunk)

INFO:root:['Pedagogía y Saberes Nº 15  2000Silvio Aristizábal Giraldo\nProfesor Universidad Pedagógica NacionalLA DIVERSIDAD ÉTNICA Y CULTURAL DE COLOMBIA:\nUN DESAFÍO PARA LA EDUCACIÓN\n1. DIVERSIDAD ÉTNICA\nY CULTURAL\nLa mayoría de los colombianos', 'cuando escucha hablar de la di-versidad étnica y cultural de lanación piensa que esta expre-sión se refiere sólo a la existen-cia de indígenas y afrocolom-bianos. Pocos entienden que laexpresión alude en algún senti-do a las diferencias entre un cos-teño, un valluno y un llanero.Muchos consideran que entrelos indígenas no hay diferenciasy para otros,  los raizales de SanAndrés y Providencia no se dis-tinguen de los chocoanos. Sinembargo, la realidad es bien di-ferente. Veamos un poco cómoes la', 'dis-tinguen de los chocoanos. Sinembargo, la realidad es bien di-ferente. Veamos un poco cómoes la situación:', '1.1. Pueblos IndígenasExisten en el país más de 80\nPueblos Indígenas que hablan66 lenguas. Cada uno de estospueblos tiene su cosmo

In [10]:
len(chunk)

94

In [11]:
chunk[7]

'Arawak, Caribe, Quechua, Chib-\ncha, Tupí-Guaraní. Hay lenguaspertenecientes a familias regio-nales como Tukano, Guahibo,Uitoto, y lenguas aisladas comoel Páez, el Guambiano, el Ka-mëntsá  o el Ticuna (Trillos,1998b).\nHay lenguas tonales como las del'

4.Create Embeddings

sentences are converted into numerical values so that they can be interpreted by the computer. 

use model_name "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
logging.info(embeddings)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:root:client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
) model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [13]:
#Exaample
model = SentenceTransformer(model_name_or_path="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
sentence_embedding = model.encode(sentences="defines culture")
logging.info(sentence_embedding)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
Batches: 100%|██████████| 1/1 [00:00<00:00, 21.10it/s]
INFO:root:[ 2.26726264e-01  2.71907240e-01  3.21822822e-01  2.00289637e-01
 -3.34747493e-01 -3.75453234e-01  3.07575524e-01 -3.38982880e-01
 -1.83738023e-01  5.34861863e-01  1.17513463e-01 -5.10601342e-01
 -1.93335742e-01 -2.09302306e-01  1.67412251e-01  7.80465528e-02
  1.66800823e-02 -3.05514246e-01 -4.22858670e-02 -9.92326066e-02
  5.86634219e-01 -1.98209472e-02  1.30212381e-01 -1.08475611e-01
 -1.25468448e-01  7.99704045e-02  1.15355864e-01 -2.75990486e-01
  3.26794147e-01 -1.78590253e-01  6.49262220e-02  4.51961994e-01
  5.59300065e-01  2.54369020e-01 -5.44850469e-01  1.23714879e-01
 -1.04890704e-01  3.07764590e-01  4.89798456e-01  4.81505871e-01
  4.03610706e-01  1.36327907e-01 -8.88490528e-02 -1.43341526e-01


In [14]:
len(sentence_embedding)

384

In [15]:
sentence_embedding

array([ 2.26726264e-01,  2.71907240e-01,  3.21822822e-01,  2.00289637e-01,
       -3.34747493e-01, -3.75453234e-01,  3.07575524e-01, -3.38982880e-01,
       -1.83738023e-01,  5.34861863e-01,  1.17513463e-01, -5.10601342e-01,
       -1.93335742e-01, -2.09302306e-01,  1.67412251e-01,  7.80465528e-02,
        1.66800823e-02, -3.05514246e-01, -4.22858670e-02, -9.92326066e-02,
        5.86634219e-01, -1.98209472e-02,  1.30212381e-01, -1.08475611e-01,
       -1.25468448e-01,  7.99704045e-02,  1.15355864e-01, -2.75990486e-01,
        3.26794147e-01, -1.78590253e-01,  6.49262220e-02,  4.51961994e-01,
        5.59300065e-01,  2.54369020e-01, -5.44850469e-01,  1.23714879e-01,
       -1.04890704e-01,  3.07764590e-01,  4.89798456e-01,  4.81505871e-01,
        4.03610706e-01,  1.36327907e-01, -8.88490528e-02, -1.43341526e-01,
       -2.07098052e-01, -5.03019579e-02,  9.49738771e-02, -6.57623122e-03,
       -3.50171745e-01,  2.91954074e-02, -1.55032337e-01, -5.08026108e-02,
       -8.95422772e-02, -

embeddings of the documents obtained are created  and save to a local database

In [16]:
knowlledge_base = FAISS.from_texts(chunk, embeddings)
logging.info(knowlledge_base)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:root:<langchain_community.vectorstores.faiss.FAISS object at 0x132d41d10>


5. Question to Document PDF

the question is assigned to a variable and the 3 paragraphs of the pdf that are most similar to the question are searched to provide an answer.

In [17]:
query = "defines culture?"
docs = knowlledge_base.similarity_search(query, 2)

In [61]:
docs

[Document(page_content='“El reconocimiento dela diversidad étnica y cultural enla Constitución supone la acep-tación de la alteridad ligada a laaceptación de multiplicidad deformas de vida y sistemas decomprensión del mundo diferen-tes de los de la cultura occiden-tal” (Cifuentes, E., Corte Consti-tucional, 1993, en Ministerio deCultura, 1998: 7).'),
 Document(page_content='de su identidad étnica. La identi-dad étnica o etnicidad se entien-de como el sentido de pertenen-cia a un grupo, a partir de laaceptación de un origen común,una historia, una cultura, una len-gua y unos valores comunes.Esta identidad se asume comoopuesta a la identidad mayorita-ria, la cual estaría asociada congrupos dominantes, a los cualesno se les aplicaría la noción deetnia. Los grupos étnicos, ade-más del reconocimiento culturaly la identidad de grupo, reclamanderechos')]

6. Create Prompt

In [78]:
role_strings = [
    ("system", "you response with rhymes or poem, and you can answer in Spanish"), 
    ("assistant", "defines culture?" ),
]
prompt = ChatPromptTemplate.from_role_strings(role_strings)
logging.info(prompt)

INFO:root:input_variables=[] messages=[ChatMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='you response with rhymes or poem, and you can answer in Spanish'), role='system'), ChatMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='defines culture?'), role='assistant')]


the obtained variables are passed to openai so that it does not return the answers according to the document: 

we pass the initially the api_key, 
the model, 
the documents and the question.

In [79]:

llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.environ["OPENAI_API_KEY"])
chain = LLMChain(llm=llm, prompt=prompt)

In [80]:
response = chain.run(input_documents=docs, question=query)
print(f"Response ChatGPT: {response}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response ChatGPT: Cultura es el alma de una nación,
su historia, tradiciones y creación.
Es el arte, la música y la danza,
que nos conecta y nos da esperanza.

Es la comida que nos alimenta el alma,
y las costumbres que nos unen con calma.
Es el idioma que nos hace comunicar,
y las creencias que nos ayudan a amar.

Cultura es diversidad y riqueza,
es la esencia de nuestra naturaleza.
Nos enseña a respetar y a valorar,
nuestro pasado y lo que vamos a crear.


7. Review cost

In [24]:
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
    print(cb)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens Used: 332
	Prompt Tokens: 287
	Completion Tokens: 45
Successful Requests: 1
Total Cost (USD): $0.0005205


8. Run app.py 




app.py is a file containing the user interface with the code functionality explained above, when opened you will be able to test it very easily.



In [67]:
!streamlit run app.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.8:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
/Users/macbook/Desktop/testPython/testlang/lib/python3.11/site-packages/langchain/embeddings/__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import HuggingFaceEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/Users/macbook/Desktop/testPython/testlang/lib/python3.11/site-packages/langchain/vectorstores/__init__.py:35: LangChainDeprecationWarning: Importing vector stores from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Pl